In [1]:
__author__ = "Lucia Seitz"
__copyright__ = "Copyright 2017, AI Research, Data Technology Centre, Volkswagen Group"
__credits__ = ["Lucia Seitz, Karolina Stosio"]
__license__ = "MIT"

import glob
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import misc
from random import shuffle
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsClassifier
import scipy as sc
%matplotlib inline

In [2]:
import edward as ed
import os
import tensorflow as tf

from edward.models import Bernoulli, Normal
from edward.util import Progbar
from observations import mnist
from scipy.misc import imsave
from tensorflow.contrib import slim

In [3]:
def generator(array, batch_size):
    """Generate batch with respect to array's first axis."""
    start = 0    # pointer to where we are in iteration
    while True:
        stop = start + batch_size
        diff = stop - array.shape[0]
        if diff <= 0:
            batch = array[start:stop]
            start += batch_size
        else:
            batch = np.concatenate((array[start:], array[:diff]))
            start = diff
        batch = batch.astype(np.float32) / 255.0    # normalize pixel intensities
        batch = np.random.binomial(1, batch)    # binarize images
        yield batch


def generative_network(z,reuse=False):
    """Generative network to parameterize generative model. It takes
    latent variables as input and outputs the likelihood parameters.
    logits = neural_network(z)
    """
    with tf.variable_scope('generative') as sc:
        with slim.arg_scope([slim.conv2d_transpose],
                                                activation_fn=tf.nn.elu,
                                                normalizer_fn=slim.batch_norm,
                                                normalizer_params={'scale': True}):
            if reuse:
                sc.reuse_variables()
            print('generative')
            print(z)
            net = tf.reshape(z, [M, 1, 1, d])
            print(net)
            net = slim.conv2d_transpose(net, 128, 4, stride=4, scope='t_conv1')
            print(net)
            net = slim.conv2d_transpose(net, 64, 5, stride=2, scope='t_conv2')
            print(net)
            net = slim.conv2d_transpose(net, 32, 5, stride=2, scope='t_conv3')
            print(net)
            net = slim.conv2d_transpose(net, 16, 5, stride=2, scope='t_conv4')
            print(net)
            net = slim.conv2d_transpose(net, 3, 5, stride=2, activation_fn=None,scope='t_conv5')
            print(net)
            net = slim.flatten(net)
            print(net)
            print('generative part completed')
    return net


def inference_network(x,reuse=False):
    """Inference network to parameterize variational model. It takes
    data as input and outputs the variational parameters.
    loc, scale = neural_network(x)
    """
    with tf.variable_scope('inference') as sc:
        with slim.arg_scope([slim.conv2d, slim.fully_connected],
                                                activation_fn=tf.nn.elu,
                                                normalizer_fn=slim.batch_norm,
                                                normalizer_params={'scale': True}):
            if reuse:
                sc.reuse_variables()
            print('inference')
            print(x)
            #net = tf.reshape(x, [M, 28, 28, 1])
            net = tf.reshape(x, [M, 64, 64, 3])
            print(net)
            net = slim.conv2d(net, 8, 5, stride=2, scope='conv1')
            print(net)
            net = slim.conv2d(net, 16, 5, stride=3, scope='conv2')
            print(net)
            net = slim.conv2d(net, 32, 5, stride=3, scope='conv3')
            print(net)
            net = slim.conv2d(net, 64, 5, stride=2, scope='conv4')
            print(net)
            net = slim.dropout(net, 0.9)
            print(net)
            net = slim.flatten(net)
            print(net)
            params = slim.fully_connected(net, d * 2, activation_fn=None, scope='fc1')
            print('params',net)

        loc = params[:, :d]
        scale = tf.nn.softplus(params[:, d:])
    print('loc',loc)
    print('scale',scale)
    print('inference part completed')
    return loc, scale

In [4]:
PATH = '/home/dlrc/r'
images = glob.glob(PATH+'/*/64px/*')
shuffle(images)

In [5]:
train_data = [misc.imread(im).flatten() for im in images if im.split('/')[-3].split('_')[1]!='blue' and misc.imread(im).shape == (64,64,3)]
test_data = [misc.imread(im).flatten() for im in images if im.split('/')[-3].split('_')[1]=='blue' and misc.imread(im).shape == (64,64,3)]

In [6]:
train_data = np.array(train_data)
test_data = np.array(test_data)

In [7]:
ed.set_seed(42)

data_dir = "/tmp/data"
out_dir = "tmp/out/" 
ckpoint_dir = "tmp/checkpoints/"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(ckpoint_dir):
    os.makedirs(ckpoint_dir)
ed.set_seed(42)
M = 128    # batch size during training
d = 10     # latent dimension

In [8]:
# DATA. MNIST batches are fed at training time.
x_test_generator = generator(test_data, M)
x_train_generator = generator(train_data, M)

# MODEL
z = Normal(loc=tf.zeros([M, d]), scale=tf.ones([M, d]))
logits = generative_network(z)
x = Bernoulli(logits=logits)

# INFERENCE
x_ph = tf.placeholder(tf.int32, [M, 64*64*3])
#x_ph = tf.placeholder(tf.int32, [M, img_x, img_y, img_chan])
loc, scale = inference_network(tf.cast(x_ph, tf.float32))
qz = Normal(loc=loc, scale=scale)

# # TEST INFERENCE
# test_x_ph = tf.placeholder(tf.int32, [M, 64*64*3])
# test_loc, test_scale = inference_network(tf.cast(test_x_ph, tf.float32),reuse=True)
# test_qz = Normal(loc=loc, scale=scale)
# test_logits = generative_network(test_qz,reuse=True)

# def kl_divergence(p, q): 
#     return tf.reduce_sum(p * tf.log(p/q))

# train_loss = kl_divergence(x_ph, q)
# test_loss = kl_divergence(p, q)

generative
RandomVariable("Normal_2/", shape=(128, 10), dtype=float32)
Tensor("generative/Reshape:0", shape=(128, 1, 1, 10), dtype=float32)
Tensor("generative/t_conv1/Elu:0", shape=(128, 4, 4, 128), dtype=float32)
Tensor("generative/t_conv2/Elu:0", shape=(128, 8, 8, 64), dtype=float32)
Tensor("generative/t_conv3/Elu:0", shape=(128, 16, 16, 32), dtype=float32)
Tensor("generative/t_conv4/Elu:0", shape=(128, 32, 32, 16), dtype=float32)
Tensor("generative/t_conv5/BatchNorm/batchnorm/add_1:0", shape=(128, 64, 64, 3), dtype=float32)
Tensor("generative/Flatten/Reshape:0", shape=(128, 12288), dtype=float32)
generative part completed
inference
Tensor("Cast:0", shape=(128, 12288), dtype=float32)
Tensor("inference/Reshape:0", shape=(128, 64, 64, 3), dtype=float32)
Tensor("inference/conv1/Elu:0", shape=(128, 32, 32, 8), dtype=float32)
Tensor("inference/conv2/Elu:0", shape=(128, 11, 11, 16), dtype=float32)
Tensor("inference/conv3/Elu:0", shape=(128, 4, 4, 32), dtype=float32)
Tensor("inference/conv4

In [9]:
variables_2_save = slim.get_variables()
saver = tf.train.Saver(variables_2_save)

In [10]:
# Bind p(x, z) and q(z | x) to the same placeholder for x.
data = {x: x_ph}
inference = ed.KLqp({z: qz}, data)
optimizer = tf.train.AdamOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer)

hidden_rep = tf.sigmoid(logits)

tf.global_variables_initializer().run()

In [14]:
### TRAINING

sess = None
n_epoch = 10000
n_iter_per_epoch = train_data.shape[0] // M
#n_epoch = 1
#n_iter_per_epoch = 3

for epoch in range(1, n_epoch + 1):
      
    print("Epoch: {0}".format(epoch))
    avg_loss = 0.0
    
    pbar = Progbar(n_iter_per_epoch)
    for t in range(1, n_iter_per_epoch + 1):
        pbar.update(t)
        x_batch = next(x_train_generator)
        info_dict = inference.update(feed_dict={x_ph: x_batch})
        avg_loss += info_dict['loss']
    
    if not sess:
        sess = ed.get_session()
    if epoch%1 == 0:
        save_path = saver.save(sess, "tmp/checkpoints/vae_"+str(epoch)+".ckpt")
    
    # Print a lower bound to the average marginal likelihood for an
    # image.
    avg_loss = avg_loss / n_iter_per_epoch
    avg_loss = avg_loss / M
    print("-log p(x) <= {:0.3f}".format(avg_loss))

    # Visualize hidden representations.
    images = hidden_rep.eval()
    for m in range(M):
        imsave(out_dir+str(m)+'.png', images[m].reshape(64, 64, 3))


Epoch: 1
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3853.891
Epoch: 2
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3866.079
Epoch: 3
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3865.801
Epoch: 4
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3872.660
Epoch: 5
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3868.597
Epoch: 6
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3879.898
Epoch: 7
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3852.153
Epoch: 8
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.054
Epoch: 9
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3869.110
Epoch: 10
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3868.313
Epoch: 11
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3864.065
Epoch: 12
9/9 [100%] ██████████████████████████████ Elapsed: 0s

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3868.007
Epoch: 191
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.433
Epoch: 192
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.616
Epoch: 193
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3852.307
Epoch: 194
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.286
Epoch: 195
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3876.278
Epoch: 196
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3862.745
Epoch: 197
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3874.727
Epoch: 198
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3864.495
Epoch: 199
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.584
Epoch: 200
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3868.336
Epoch: 201
9/9 [100%] ██████████████████████████████ E

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3861.902
Epoch: 379
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3864.802
Epoch: 380
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3856.197
Epoch: 381
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3866.687
Epoch: 382
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3859.261
Epoch: 383
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3855.673
Epoch: 384
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3849.907
Epoch: 385
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3864.155
Epoch: 386
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3869.592
Epoch: 387
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.154
Epoch: 388
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3861.388
Epoch: 389
9/9 [100%] ██████████████████████████████ E

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3864.718
Epoch: 567
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3869.819
Epoch: 568
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.247
Epoch: 569
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3858.700
Epoch: 570
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3865.129
Epoch: 571
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3865.085
Epoch: 572
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3858.800
Epoch: 573
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3850.048
Epoch: 574
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3864.583
Epoch: 575
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3850.446
Epoch: 576
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3862.591
Epoch: 577
9/9 [100%] ██████████████████████████████ E

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3850.010
Epoch: 755
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3862.106
Epoch: 756
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3868.814
Epoch: 757
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3858.281
Epoch: 758
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.264
Epoch: 759
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3868.481
Epoch: 760
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3856.723
Epoch: 761
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.992
Epoch: 762
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3861.535
Epoch: 763
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3855.931
Epoch: 764
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3863.401
Epoch: 765
9/9 [100%] ██████████████████████████████ E

...

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3851.241
Epoch: 9221
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3841.972
Epoch: 9222
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3851.693
Epoch: 9223
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3839.558
Epoch: 9224
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3853.774
Epoch: 9225
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3850.899
Epoch: 9226
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3856.554
Epoch: 9227
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3852.545
Epoch: 9228
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3870.561
Epoch: 9229
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3837.071
Epoch: 9230
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3851.485
Epoch: 9231
9/9 [100%] █████████████████████

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3856.552
Epoch: 9407
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3860.423
Epoch: 9408
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3849.433
Epoch: 9409
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3855.740
Epoch: 9410
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.264
Epoch: 9411
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3846.683
Epoch: 9412
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.225
Epoch: 9413
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3847.471
Epoch: 9414
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3853.053
Epoch: 9415
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3841.869
Epoch: 9416
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3852.380
Epoch: 9417
9/9 [100%] █████████████████████

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3849.587
Epoch: 9593
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3838.914
Epoch: 9594
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3857.903
Epoch: 9595
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3853.981
Epoch: 9596
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.035
Epoch: 9597
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3858.630
Epoch: 9598
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3869.963
Epoch: 9599
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3843.333
Epoch: 9600
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3847.815
Epoch: 9601
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3856.595
Epoch: 9602
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3848.485
Epoch: 9603
9/9 [100%] █████████████████████

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3845.338
Epoch: 9779
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.946
Epoch: 9780
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3851.950
Epoch: 9781
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3848.025
Epoch: 9782
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3850.305
Epoch: 9783
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3847.249
Epoch: 9784
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3845.715
Epoch: 9785
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3846.972
Epoch: 9786
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3848.907
Epoch: 9787
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3857.257
Epoch: 9788
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3857.149
Epoch: 9789
9/9 [100%] █████████████████████

9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3853.075
Epoch: 9965
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3858.575
Epoch: 9966
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.931
Epoch: 9967
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3854.124
Epoch: 9968
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3868.941
Epoch: 9969
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3840.421
Epoch: 9970
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3849.732
Epoch: 9971
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3856.098
Epoch: 9972
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3844.858
Epoch: 9973
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3859.569
Epoch: 9974
9/9 [100%] ██████████████████████████████ Elapsed: 0s
-log p(x) <= 3844.156
Epoch: 9975
9/9 [100%] █████████████████████